## Nodes

In [1]:
total_nodes = int(input("Enter no.of Nodes: "))

Enter no.of Nodes: 10


## Cores

In [2]:
cores_per_node = int(input("Enter no.of Cores per Node: "))

Enter no.of Cores per Node: 16


## RAM

In [3]:
ram_per_node = int(input("Enter size of RAM (GB) per Node:"))

Enter size of RAM (GB) per Node:64


## Deployment Mode

In [4]:
deployment_mode = input("Enter the deployment mode (Cluster/Client): ")

Enter the deployment mode (Cluster/Client): client


### Calculation

In [5]:
def bold(text):
    return "\033[1m" + text + "\033[0m"

In [6]:
print("Total Nodes: " + str(total_nodes) + " (1 Master node + " + str(total_nodes-1) + " Worker nodes)")

Total Nodes: 10 (1 Master node + 9 Worker nodes)


In [7]:
executor_cores = 5 # Optimal usage, 5 cores per executor
yarn_cores = 1 # Yarn Daemon, 1 core per node
print(bold("--executor-cores " + str(executor_cores)))

--executor-cores 5


In [8]:
available_cores_per_node = cores_per_node - yarn_cores
print("Available Cores per Node: " + str(available_cores_per_node))

total_cores_cluster = available_cores_per_node*total_nodes
print("Total Cores in Cluster: " + str(total_cores_cluster) + " (" + str(available_cores_per_node) + " Cores per Node)")

Available Cores per Node: 15
Total Cores in Cluster: 150 (15 Cores per Node)


In [9]:
num_executors = int(total_cores_cluster/executor_cores)

if deployment_mode.lower() == 'cluster':
    # Leaving 1 executor for ApplicationManager/Driver
    num_executors = num_executors-1

print(bold("--num-executors " + str(num_executors)))

--num-executors 30


In [10]:
default_parallelism = num_executors*executor_cores*2

print("Default Parallelism: " + str(default_parallelism))

Default Parallelism: 300


In [11]:
import math


executors_per_node = int(available_cores_per_node/executor_cores)
print("Number of Executors per Node: " + str(executors_per_node))

total_ram_per_executor = int(ram_per_node/executors_per_node)
# Heap memory overhead is 10%
heap_memory_overhead = math.ceil(total_ram_per_executor*0.1)

print("Total memory (GB) per Executor: " + str(total_ram_per_executor))
print("Heap memory (GB) overhead: " + str(heap_memory_overhead))

Number of Executors per Node: 3
Total memory (GB) per Executor: 21
Heap memory (GB) overhead: 3


In [12]:
executor_memory = total_ram_per_executor - heap_memory_overhead
print(bold("--executor-memory " + str(executor_memory) + "G"))

--executor-memory 18G


In [13]:
spark_configuration = {
            'spark.driver.cores': executor_cores,
            'spark.driver.memory': str(executor_memory)+'g',
            'spark.driver.memoryOverhead': str(heap_memory_overhead)+'g',
            'spark.executor.cores': executor_cores,
            'spark.executor.memory': str(executor_memory)+'g',
            'spark.executor.memoryOverhead': str(heap_memory_overhead)+'g',
            'spark.executor.instances': num_executors,
            'spark.default.parallelism': default_parallelism,
            'spark.sql.shuffle.partitions': default_parallelism,
        }

import json
print(json.dumps(spark_configuration, indent=2))

{
  "spark.driver.cores": 5,
  "spark.driver.memory": "18g",
  "spark.driver.memoryOverhead": "3g",
  "spark.executor.cores": 5,
  "spark.executor.memory": "18g",
  "spark.executor.memoryOverhead": "3g",
  "spark.executor.instances": 30,
  "spark.default.parallelism": 300,
  "spark.sql.shuffle.partitions": 300
}


Checked: 

https://spoddutur.github.io/spark-notes/distribution_of_executors_cores_and_memory_for_spark_application.html

https://codepen.io/mfeet/pen/eeMJZr

https://github.com/KanchiShimono/scopt


To Check:



https://blogs.perficient.com/2020/08/25/key-components-calculations-for-spark-memory-management/

https://wganesh.medium.com/spark-cluster-sizing-ed70cd7eec33

Must Check after above links:

http://spark-configuration.luminousmen.com/

https://stackoverflow.com/questions/65822771/how-to-calculate-the-executors-cores-and-memory-based-on-a-given-input-file-siz

Least Priority:
https://blog.cloudera.com/how-to-tune-your-apache-spark-jobs-part-1/

https://blog.cloudera.com/how-to-tune-your-apache-spark-jobs-part-2/